In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, Math, Latex

In [2]:
from sympy import *
from sympy.vector import CoordSys3D
import numpy as np
from functools import reduce
from sympy.abc import lamda as lmd

In [3]:
def print_eq(eq, *exprs):
    if type(eq) == str:
        eq = Symbol(eq)
    out = '$$ {} ' + '= {}' * len(exprs) + '$$'
    exprs=list(exprs)
    for i in range(len(exprs)):
        if isinstance(exprs[i], (np.ndarray,)):
            exprs[i] = pmatrix(exprs[i])
        elif type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    display(Latex(out.format(latex(eq), *exprs)))

def print_system(*exprs, collab=False): 
    '''
    collab говнюк не хочет нормально системы уравнений отоброжать, 
    если вы в Jupyter, то замените в аргументе collab True на False'''
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    exprs_latex = r'\\'.join(exprs)
    if collab:
        system = r'Начало системы\\' + exprs_latex + r'\\Конец_системы\\'
    else:
        system = \
        r'''\begin{equation*}
    \begin{cases}''' \
        + exprs_latex + \
        r'''\end{cases}
    \end{equation*}
        '''

    print_eq(system)
    
def to_s(i):
    return Symbol(str(i))

def pmatrix(array):
    if type(array) != Matrix:
        array = Matrix(array)
    return latex(array)[6:-7].replace('matrix', 'pmatrix') + ''

# Задание 1

In [4]:
bar_a = Symbol(r'\bar a')
bar_b = Symbol(r'\bar b')

In [5]:
variants = [
    [
        [0, 4, 7, 2],
        [-2, 4, 3, -1],
        [4, -4, 1, 4],
        [2, 4, 11, 5]
    ],
    [
        [1, 2, 3],
        [1, 3, 9],
        [1, 5, 17]
    ],
    [
        [5, 1, -1, 2],
        [4, -1, -1, 6],
        [-3, 3, 1, -10],
        [6, 3, -1, -2]
    ]
]

In [6]:
a = variants[2]
a = [np.array(el) for el in a]
n = len(a)
f = np.dot

In [7]:
b = [
    a[0]
]

print_eq(f'{bar_b}_1', f'{bar_a}_1')

for i in range(1, n):
    b_gen_view = f'{bar_b}_{i + 1}={bar_a}_{i + 1}'
    b_solved_view = f'{bar_b}_{i + 1}={pmatrix(a[i])}'
    p1 = a[i]
    b_n = p1
    
    
    for j in range(1, i + 1):
        b_gen_view += f'-\\frac{{({bar_a}_{i + 1};{bar_b}_{j})}}{{({bar_b}_{j};{bar_b}_{j})}}{bar_b}_{j}'
    print_eq(b_gen_view)    
    
    for j in range(1, i + 1):
        err = None
        p_n = Rational(
            f(a[i], b[j - 1]),
            f(b[j - 1], b[j - 1])
        )
        if p_n == nan:
            err = ZeroDivisionError('Получаем деление на 0')
            break
            
        b_solved_view += '-' + latex(p_n)
        b_solved_view += r'\cdot' + pmatrix(b[j - 1])
        p_n = p_n * b[j - 1]
        b_n = b_n - p_n
        
    b_solved_view += '=' + pmatrix(b_n)
    
    if not err:
        nok = 1
        for el in b_n:
            nok = np.gcd(nok, el)
        b_n = b_n / nok

        nod = reduce(np.gcd, b_n)
        b_n = b_n / (1 if not nod else nod)

    b.append(b_n)
    
    print_eq(b_solved_view)
    print_eq(f'{bar_b}_{i + 1}', (b_n))
    
    print('Проверка:')
    for j in range(i):
        print_eq(f'({bar_b}_{{{j + 1}}};{bar_b}_{i + 1})', f(b[j], b[i]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Проверка:


...

<IPython.core.display.Latex object>

Проверка:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
b_b_sqrts = []
for i in range(len(b)):
    b_b_sqrt = sqrt(f(b[i], b[i]))
    b_b_sqrts.append(b_b_sqrt)
    print_eq(f'|{bar_b}_{i + 1}|', b_b_sqrt)

for i in range(len(b)):
    if b_b_sqrts[i]:
        h = b[i] / b_b_sqrts[i]
    else:
        h = b[i] * 0
    print_eq(f'|\\bar h_{{{i + 1}}}|', pmatrix(h))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Задание 2

In [9]:
integral_from = 1
integral_to = -1

In [10]:
print_eq(f'(f(x);g(x))=\int_{{{integral_to}}}^{{{integral_from}}} (f(x);g(x)) \,dx')

<IPython.core.display.Latex object>

In [11]:
t = Symbol('t')
task_symbol = Symbol('f')

In [12]:
exprs = [
    3*t + t ** 2,
    1 + 3 * t + 2 * t ** 2,
    1 + 5 * t + 3 * t ** 2,
]

for i in range(len(exprs)):
    expr = exprs[i]
    print_eq(latex(task_symbol) + f"_{i + 1}", expr)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [13]:
Integral(exprs[0] * exprs[1], (t, integral_to, integral_from))

Integral((t**2 + 3*t)*(2*t**2 + 3*t + 1), (t, -1, 1))

In [14]:
n = len(exprs)
f = lambda x, y: Integral(x * y, (t, integral_to, integral_from)).doit()

In [15]:
f(exprs[0], exprs[1])

112/15

In [23]:
b = [
    exprs[0]
]

print_eq(f'{bar_b}_1', f'{task_symbol}_1')

for i in range(1, n):
    b_gen_view = f'b_{i + 1}={task_symbol}_{i + 1}'
    b_solved_view = f'b_{i + 1}=({latex(exprs[i])})'
    p1 = exprs[i]
    b_n = p1
    
    
    for j in range(1, i + 1):
        b_gen_view += f'-\\frac{{({task_symbol}_{i + 1};{bar_b}_{j})}}{{({bar_b}_{j};{bar_b}_{j})}}{bar_b}_{j}'
    print_eq(b_gen_view)    
    
    for j in range(1, i + 1):
        err = None
        p_n = Rational(
            f(exprs[i], b[j - 1]),
            f(b[j - 1], b[j - 1])
        )
        if p_n == nan:
            err = ZeroDivisionError('Получаем деление на 0')
            break
            
        b_solved_view += '-' + latex(p_n)
        b_solved_view += r'\cdot ' + f'({latex(b[j - 1])})'
        p_n = p_n * b[j - 1]
        b_n = b_n - p_n
        
    b_solved_view += '=' + latex(b_n)
    
    if not err:
        pass

    b.append(b_n)
    
    print_eq(b_solved_view)
    
    print('Проверка:')
    for j in range(i):
        print_eq(f'({bar_b}_{{{j + 1}}};{bar_b}_{i + 1})', f(b[j], b[i]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Проверка:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Проверка:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [27]:
b_b_sqrts = []
for i in range(len(b)):
    b_b_sqrt = sqrt(f(b[i], b[i]))
    b_b_sqrts.append(b_b_sqrt)
    print_eq(f'|b_{i + 1}|', b_b_sqrt)

for i in range(len(b)):
    if b_b_sqrts[i]:
        h = b[i] / b_b_sqrts[i]
    else:
        h = b[i] * 0
    print_eq(f'|h_{{{i + 1}}}|', expand(h))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>